[Read here](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

# Saving
* Model
* State Dictionary

### torch.save(args, PATH) 
### it can be used to save tensor models or <span style='color:cyan'>any dict</span> as parameter for saving.
torch.save use python <span style='color:cyan'>pickle module</span> to serialize the objects and saves them.

## Model

This is the lazy method. The disadvantage is that serialized data is bound to the specific classes.

torch.save (model, PATH)

model = torch.load(PATH)
model.eval()

<span style='color:cyan'>while loading, the model class must be defined somewhere. Pickle does not save the model class itself. Rather, it saves a path to the file containing the class, which is used during load time. Because of this, your code can break in various ways when used in other projects or after refactors.</span>

## State Dictionary <span style='color:cyan'>(Recommanded Way)</span>
save only the parameters


torch.save(model.state_dict(), PATH)

model = Model(*args,**kwargs)

model.load_state_dict(torch.load(PATH))

model.eval()

<span style='color:cyan'>model must be created again with parameters</span>

In [61]:
import torch
import torch.nn as nn

In [62]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model,self).__init__()
        self.linear = nn.Linear(in_features=n_input_features, out_features=1)
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [63]:
model = Model(n_input_features=6)

## Saving and loading the whole model
<span style='color:cyan'> file must be ended with ".pth" or ".pt".</span>

loading the model from other file in load_model.py<span style='color:cyan'> which will give you error due to window 10</span>

In [64]:
FILE = 'whole_model.pt'
torch.save(model, FILE)

In [65]:
loaded_model = torch.load(FILE)
loaded_model.eval()

for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0268, -0.2908, -0.0573,  0.0077, -0.0466,  0.2516]],
       requires_grad=True)
Parameter containing:
tensor([-0.0584], requires_grad=True)


## Saving the state_dict
loading state_dict in load_state_dict.py

we have to <span style='color:cyan'>rebuild the model class.</span>

In [66]:
FILE = "state_dict.pt"
torch.save(model.state_dict(),FILE)

In [67]:
loaded_model = Model(n_input_features=6)
loaded_model.load_state_dict(torch.load(FILE))
loaded_model.eval()

for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0268, -0.2908, -0.0573,  0.0077, -0.0466,  0.2516]],
       requires_grad=True)
Parameter containing:
tensor([-0.0584], requires_grad=True)


# Saving Checkpoint during Training

load_check_point will also be in load_state_dict.py

In [68]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1]}]}


In [69]:
checkpoint = {
    'epoch': 90,
    'model_state':model.state_dict(),
    'optim_state':optimizer.state_dict()
}

torch.save(checkpoint,'checkpoint.pt')

In [70]:
loaded_checkpoint = torch.load('checkpoint.pt')
epoch = loaded_checkpoint['epoch']

### **we have to recreate and address the model and optimizer**
<span style='color:cyan'>we do not need the correct learning rate</span>

In [71]:
loaded_model = Model(n_input_features=6)
# different learning rate
optimizer = torch.optim.SGD(model.parameters(),lr=0.0)

In [72]:
loaded_model.load_state_dict(loaded_checkpoint['model_state'])
optimizer.load_state_dict(loaded_checkpoint['optim_state'])
# check that the learning rate is loaded now.
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1]}]}


## Saving on GPU, Load on CPU

saving

In [ ]:
device = torch.device('cuda')
model.to(device)
torch.save(model.state_dict(),PATH)

loading

In [73]:
device = torch.device('cpu')
model = Model(*args,**kwargs)
model.load_state_dict(torch.load(PATH,map_location=device))

NameError: name 'args' is not defined

## Save on GPU, Load on GPU

saving

In [ ]:
device = torch.device('cuda')
model.to(device)
torch.save(model.state_dict(),PATH)

loading

In [ ]:
device = torch.device('cuda')
model = Model(*args,**kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)

## Save on CPU, Load on GPU

saving

In [ ]:
torch.save(model.state_dict(),PATH)

loading

In [ ]:
device = torch.device('cuda')
model = Model(*args,**kwargs)
model.load_state_dict(torch.load(PATH,map_location='cuda:0'))
model.to(device)